##  Sentinel-3 OLCI Level-2 Tiling

### Service Definition

In [ ]:
service = dict([('title', 'Sentinel-3 OLCI level-2 tiling'),
                ('abstract', 'This service takes as input one or more Sentinel-3 OLCI Level 2 (OL_2_LFR____) products and tiles'),
                ('identifier', 'ewf-wfp-03-01-01')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [ ]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____)'),
                        ('abstract', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____) catalogue reference'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?uid=S3B_OL_2_LFR____20200825T102904_20200825T103204_20200825T122020_0179_042_336_2340_LN1_O_NR_002'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [ ]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '5'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data/s3'

In [ ]:
input_catalog = '/workspace/data/s3/catalog.json'

### Workflow

#### Import the packages

In [ ]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'

In [ ]:
import os
import sys
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
sys.path.append('.')
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
from tiling import s3_tiles
import glob
import numpy as np
gdal.UseExceptions()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [ ]:
collection = next(cat.get_children())

In [ ]:
item = next(collection.get_items())

In [ ]:
item

### Import Sentinel-3 SLSTR product

In [ ]:
operators = ['Read', 
             'Reproject',
             'Write']

In [ ]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_olci'

In [ ]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Reproject=reproject,
           Write=write)

In [141]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds
    x_min, y_min, x_max, y_max = tile.tile.buffer(0.02).bounds

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)


    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['NDVI', 'OGVI', 'OCTI', 'Land mask', 'Cloud mask', 'OGVI fail mask']

    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)

        srcband.SetDescription(band_names[index])
        # set noData for ndvi ogvi and otci as 255
        if index <= 2:
            
            srcband.SetNoDataValue(255)
    
    ds.FlushCache()

    ds = None
    
    ###Drop generated tiles where land is less than Threshold=0.05
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    land_mask=ds.GetRasterBand(4).ReadAsArray()
    if np.count_nonzero(land_mask)/land_mask.size < 0.05 : 
        
        os.remove('{}.tif'.format(output_tile_name))
    else: 
        ndvi_tmp=ds.GetRasterBand(1).ReadAsArray()
        ogvi_tmp=ds.GetRasterBand(2).ReadAsArray()
        otci_tmp=ds.GetRasterBand(3).ReadAsArray()
        # If land_mask=0 set data to noData
    
        masked_data = lambda x,y: 255 if y==0  else x
        vfunc_masked = np.vectorize(masked_data, otypes=[np.float])
    
        updated_ndvi_data = vfunc_masked(ndvi_tmp, land_mask)
        updated_ogvi_data = vfunc_masked(ogvi_tmp, land_mask)
        updated_otci_data = vfunc_masked(otci_tmp, land_mask)
    
        ds.GetRasterBand(1).WriteArray(updated_ndvi_data)
        ds.GetRasterBand(2).WriteArray(updated_ogvi_data)
        ds.GetRasterBand(3).WriteArray(updated_otci_data)

        with open(output_tile_name + '.properties', 'w') as file:

            file.write('title=Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                                  tile.level, 
                                                                  tile.col, 
                                                                  tile.row))

            file.write('date={}/{}\n'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ'), 
                                             item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ')))

            file.write('geometry={0}'.format(tile.tile.wkt))


    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

            if os.path.exists(f):

                os.remove(f)

    ds = None
    return True
    

### Tiling

In [ ]:
bands = [os.path.join('s3_olci.data', '{}.img'.format(band)) for band in ['OGVI', 'OTCI', 'RC681', 'RC865', 'LQSF']]
        
s3_data = read_s3(bands)
        

In [ ]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [ ]:
ogvi = s3_data[:,:,0]
otci= s3_data[:,:,1]
red = s3_data[:,:,2]
nir = s3_data[:,:,3]
lqsf = s3_data[:,:,4]
#otci_quality_flags=s3_data[:,:,5]

In [102]:
mask = get_mask('LAND', lqsf)

In [103]:
cloud_mask =  get_mask('CLOUD', lqsf)

In [105]:
ogvi_fail_mask =  get_mask('OGVI_FAIL', lqsf)
#if mask=1 then ogvi=255

In [121]:
#Calculate NDVI
ndvi_lambda = lambda x,y,z,w,v: 255 if(x+y)==0 or z or w or v==0 else  (x-y)/float(x+y)
vfunc_ndvi = np.vectorize(ndvi_lambda, otypes=[np.float])

ndvi=vfunc_ndvi(nir, red, ogvi_fail_mask ,cloud_mask , mask)


In [128]:
output_name = 'temp.tif'

In [129]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       ogvi.shape[1], 
                       ogvi.shape[0], 
                       6, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(ndvi)
output.GetRasterBand(2).WriteArray(ogvi)
output.GetRasterBand(3).WriteArray(otci)
output.GetRasterBand(4).WriteArray(mask)
output.GetRasterBand(5).WriteArray(cloud_mask)
output.GetRasterBand(6).WriteArray(ogvi_fail_mask)

output.FlushCache()

output = None

In [136]:
tiles = s3_tiles(shape(item.geometry), int(tiling_level['value']))


In [137]:
tiles

,col,row,level,s3_tile,tile
0,30,21,5,"POLYGON ((-5.62500 30.32187, -5.92126 30.37690...","POLYGON ((-5.62500 28.12500, -5.62500 33.75000..."
1,31,21,5,"POLYGON ((0.00000 29.15251, -0.35698 29.23620,...","POLYGON ((0.00000 28.12500, 0.00000 33.75000, ..."
2,32,22,5,"POLYGON ((3.85818 39.37500, 2.94420 36.94130, ...","POLYGON ((5.62500 33.75000, 5.62500 39.37500, ..."
3,30,22,5,"POLYGON ((-5.62500 33.75000, -11.25000 33.7500...","POLYGON ((-5.62500 33.75000, -5.62500 39.37500..."
4,31,22,5,"POLYGON ((0.00000 33.75000, -5.62500 33.75000,...","POLYGON ((0.00000 33.75000, 0.00000 39.37500, ..."
5,29,22,5,"POLYGON ((-12.65983 33.75000, -12.60190 34.095...","POLYGON ((-11.25000 33.75000, -11.25000 39.375..."
6,29,23,5,"POLYGON ((-11.73834 39.37500, -11.32170 41.976...","POLYGON ((-11.25000 39.37500, -11.25000 45.000..."
7,30,23,5,"POLYGON ((-11.25000 41.97063, -10.51000 41.908...","POLYGON ((-5.62500 39.37500, -5.62500 45.00000..."
8,32,23,5,"POLYGON ((0.00000 40.38559, 0.00000 40.38559, ...","POLYGON ((5.62500 39.37500, 5.62500 45.00000, ..."
9,31,23,5,"POLYGON ((-5.62500 41.32892, -5.56774 41.32100...","POLYGON ((0.00000 39.37500, 0.00000 45.00000, ..."


In [142]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

In [143]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_olci.data')
os.remove('s3_olci.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.